In [50]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv(r"C:\Users\Lucy\Downloads\Online_Retail.zip")

In [51]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [52]:
#small data wrangling task
#make sure we dont have any quantity as 0

df['total_revenue'] = df['Quantity']*df['UnitPrice']
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
df = df[df['Quantity'] >0]
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [64]:
#Here, you are going to perform the following operations:

#Calculate the number of days between the present date and the date of last purchase from each customer.
#Calculate the number of orders for each customer.
#Calculate sum of purchase price(total revenue) for each customer.

max_date = df['InvoiceDate'].max()
df1 = df.groupby('CustomerID').agg({'InvoiceDate': lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,
                                       'InvoiceNo': lambda num: len(num),
                                        'Quantity': lambda quant: quant.sum(),
                                        'total_revenue': lambda price: price.sum()})
df1

,InvoiceDate,InvoiceNo,Quantity,total_revenue
CustomerID,,,,
12346.0,0,1,74215,77183.60
12347.0,365,182,2458,4310.00
12348.0,282,31,2341,1797.24
12349.0,0,73,631,1757.55
12350.0,0,17,197,334.40
...,...,...,...,...
18280.0,0,10,45,180.60
18281.0,0,7,54,80.82
18282.0,118,12,103,178.05


In [54]:
max_date

Timestamp('2011-12-09 12:50:00')

In [67]:
#change the name of the columns and insure that we dont have 0 quantity 
df1.columns = ['age','num_transactions','quantity','total_revenue']


In [68]:
df1.head()

,age,num_transactions,quantity,total_revenue
CustomerID,,,,
12346.0,0,1,74215,77183.60
12347.0,365,182,2458,4310.00
12348.0,282,31,2341,1797.24
12349.0,0,73,631,1757.55
12350.0,0,17,197,334.40


In [72]:
#create the average order value
df1['AOV'] =df['total_revenue']/df1['num_transactions']
df1


,age,num_transactions,quantity,total_revenue,AOV
CustomerID,,,,,
12346.0,0,1,74215,77183.60,1.650000
12347.0,365,182,2458,4310.00,0.041209
12348.0,282,31,2341,1797.24,0.053226
12349.0,0,73,631,1757.55,0.017123
12350.0,0,17,197,334.40,0.073529
...,...,...,...,...,...
18280.0,0,10,45,180.60,2.010000
18281.0,0,7,54,80.82,3.600000
18282.0,118,12,103,178.05,1.325000


In [76]:
# lets calcualate the purchase frequency 
purchase_freq = sum(df1['num_transactions'])/len(df1)
purchase_freq

91.70868863793501

In [77]:
# lets calcuate the repeate rate
repeat_rate = df1[df1['num_transactions']>1].shape[0]/df1.shape[0]
repeat_rate

0.9834063148190827

In [79]:
# lets calcuate the churn rate
churn_rate = 1-repeat_rate
churn_rate

0.016593685180917306

In [80]:
#lets add the profit margin 
#i choose 10%.. you can choose any number

df1['profit_margin'] = df1['total_revenue']*.10
df1.head()

,age,num_transactions,quantity,total_revenue,AOV,profit_margin
CustomerID,,,,,,
12346.0,0,1,74215,77183.60,1.650000,7718.360
12347.0,365,182,2458,4310.00,0.041209,431.000
12348.0,282,31,2341,1797.24,0.053226,179.724
12349.0,0,73,631,1757.55,0.017123,175.755
12350.0,0,17,197,334.40,0.073529,33.440


In [82]:
# lets put the final equation 
df1['CLTV'] =((df1['AOV']*purchase_freq)/churn_rate)*.10
df1

,age,num_transactions,quantity,total_revenue,AOV,profit_margin,CLTV
CustomerID,,,,,,,
12346.0,0,1,74215,77183.60,1.650000,7718.360,911.909167
12347.0,365,182,2458,4310.00,0.041209,431.000,22.774954
12348.0,282,31,2341,1797.24,0.053226,179.724,29.416425
12349.0,0,73,631,1757.55,0.017123,175.755,9.463565
12350.0,0,17,197,334.40,0.073529,33.440,40.637663
...,...,...,...,...,...,...,...
18280.0,0,10,45,180.60,2.010000,18.060,1110.871167
18281.0,0,7,54,80.82,3.600000,8.082,1989.620000
18282.0,118,12,103,178.05,1.325000,17.805,732.290694
